In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from fairdata import FairData
from fairopt import FairOptimization

In [2]:
df_raw_1 = pd.read_excel('data/Fintech-fairnessJun2020.xlsx', sheet_name='Approved&Default')
df_raw_1

,customer_id,loan_request_id,loan_transferred_date,def_flag,age,noofconnections,noofapps,noofsms,noofcontacts,gender
0,144,199.0,2016-04-18 10:34:00,0.0,29.9315,401.0,19.0,472.0,358.0,NaN
1,144,1658.0,2016-06-30 10:04:00,1.0,29.9315,401.0,19.0,472.0,358.0,NaN
2,144,1039.0,2016-06-03 02:18:00,0.0,29.9315,401.0,19.0,472.0,358.0,NaN
3,144,417.0,2016-05-04 10:18:00,0.0,29.9315,401.0,19.0,472.0,358.0,NaN
4,150,218.0,2016-04-19 14:34:00,0.0,32.0740,811.0,10.0,584.0,1820.0,m
...,...,...,...,...,...,...,...,...,...,...
383756,3200000,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,m
383757,3200000,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,m
383758,3200000,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,m
383759,3200000,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,m


In [3]:
df_raw_2 = pd.read_excel('data/Fintech-fairnessJun2020.xlsx', sheet_name='Rejected')
df_raw_2

,customer_id,age,noofconnections,noofapps,noofsms,noofcontacts,loan_request_initial_id,gender
0,185,30.6932,0.0,54.0,3646.0,217.0,12116,m
1,237,30.1973,1556.0,97.0,302.0,589.0,145910,m
2,290,47.6575,511.0,4.0,7.0,25996.0,135,m
3,769,47.4247,249.0,18.0,366.0,565.0,193588,m
4,803,31.3452,109.0,39.0,719.0,226.0,1808,m
...,...,...,...,...,...,...,...,...
49979,3200000,26.3123,0.0,21.0,NaN,209.0,711504,m
49980,3200000,25.5945,384.0,30.0,NaN,663.0,711581,m
49981,3200000,24.1973,938.0,NaN,NaN,0.0,711596,m
49982,3200000,20.1370,0.0,98.0,NaN,409.0,711610,m


In [4]:
df_raw_1['reward'] = 2 * (0.5 - df_raw_1['def_flag'])
df_raw_2['reward'] = 0
df_raw_1['approved_dum'] = 1
df_raw_2['approved_dum'] = 0
df_raw = pd.concat([
    df_raw_1.drop(['loan_transferred_date', 'def_flag'], axis=1),
    df_raw_2.rename(columns={'loan_request_initial_id': 'loan_request_id'}),
]).rename(columns={
    'noofconnections': 'connections',
    'noofapps': 'apps',
    'noofsms': 'sms',
    'noofcontacts': 'contacts',
}).astype({
    'customer_id': 'Int64',
    'loan_request_id': 'Int64',
    'age': 'float',
    'connections': 'float',
    'apps': 'float',
    'sms': 'float',
    'contacts': 'float',
    'approved_dum': 'Int64',
    'reward': 'Int64',
})
df_raw


,customer_id,loan_request_id,age,connections,apps,sms,contacts,gender,reward,approved_dum
0,144,199,29.9315,401.0,19.0,472.0,358.0,NaN,1,1
1,144,1658,29.9315,401.0,19.0,472.0,358.0,NaN,-1,1
2,144,1039,29.9315,401.0,19.0,472.0,358.0,NaN,1,1
3,144,417,29.9315,401.0,19.0,472.0,358.0,NaN,1,1
4,150,218,32.0740,811.0,10.0,584.0,1820.0,m,1,1
...,...,...,...,...,...,...,...,...,...,...
49979,3200000,711504,26.3123,0.0,21.0,NaN,209.0,m,0,0
49980,3200000,711581,25.5945,384.0,30.0,NaN,663.0,m,0,0
49981,3200000,711596,24.1973,938.0,NaN,NaN,0.0,m,0,0
49982,3200000,711610,20.1370,0.0,98.0,NaN,409.0,m,0,0


In [5]:
df_csv = pd.read_csv('data/Cashe_information.csv').rename(columns={
    'AGE': 'age',
}).astype({
    'customer_id': 'Int64',
    'loan_request_id': 'Int64',
    'age': 'float',
    'connections': 'float',
    'apps': 'float',
    'sms': 'float',
    'contacts': 'float',
    'approved_dum': 'Int64',
})
df_csv

,customer_id,loan_request_id,loan_amount,def_flag,age,salary,referal,referer,connections,apps,...,highschool,college,googleplus_status,sales,dating,ios,travel_app,finsavy_app,socialconnect_app,mloan_app
0,320,16,30000.0,0.0,34.0466,85449.0,0.0,0.0,25.0,NaN,...,1.0,0.0,0.0,0.0,0.0,1,1,0,0,0
1,301,18,10000.0,0.0,45.2301,42603.0,0.0,1.0,1050.0,56.0,...,1.0,0.0,0.0,0.0,0.0,0,1,1,1,0
2,302,20,20000.0,0.0,53.7123,44783.0,0.0,0.0,NaN,35.0,...,1.0,0.0,0.0,0.0,0.0,0,1,0,1,0
3,286,21,5000.0,0.0,43.3644,38500.0,0.0,0.0,186.0,22.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0
4,302,19,20000.0,0.0,53.7123,44783.0,0.0,0.0,NaN,35.0,...,1.0,0.0,0.0,0.0,0.0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417670,549179,<NA>,25000.0,NaN,47.5616,25000.0,NaN,NaN,4290.0,49.0,...,0.0,0.0,NaN,NaN,NaN,0,0,0,0,0
417671,1984639,<NA>,11000.0,NaN,40.5452,27000.0,0.0,0.0,NaN,62.0,...,1.0,0.0,1.0,NaN,NaN,0,0,0,0,0
417672,776459,<NA>,5000.0,NaN,22.5205,15000.0,NaN,NaN,36.0,NaN,...,0.0,0.0,NaN,NaN,NaN,1,0,0,0,0
417673,1865368,<NA>,5000.0,NaN,26.4164,25000.0,0.0,0.0,4976.0,48.0,...,1.0,0.0,0.0,NaN,NaN,0,0,0,0,0


In [6]:
df_all = pd.merge(
    df_raw.dropna(),
    df_csv.loc[:, ['loan_request_id', 'salary', 'loan_amount', 'CIBIL']].dropna(), 
    how='left', on='loan_request_id').dropna()
df_all

,customer_id,loan_request_id,age,connections,apps,sms,contacts,gender,reward,approved_dum,salary,loan_amount,CIBIL
0,150,218,32.0740,811.0,10.0,584.0,1820.0,m,1,1,40000.0,5000.0,707.0
1,150,97964,32.0740,811.0,10.0,584.0,1820.0,m,-1,1,40000.0,40000.0,707.0
2,150,66,32.0740,811.0,10.0,584.0,1820.0,m,1,1,40000.0,5000.0,707.0
3,150,2312,32.0740,811.0,10.0,584.0,1820.0,m,1,1,40000.0,5000.0,707.0
4,150,3754,32.0740,811.0,10.0,584.0,1820.0,m,1,1,40000.0,5000.0,707.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
261924,1600000,282451,18.3178,1.0,24.0,121.0,171.0,m,0,0,115000.0,200000.0,747.0
261958,1600000,283249,30.0904,736.0,57.0,1638.0,862.0,m,0,0,50251.0,101000.0,663.0
261970,1600000,283473,18.6904,196.0,37.0,1328.0,203.0,m,0,0,68328.0,137000.0,745.0
261987,1600000,283850,27.9096,9.0,48.0,237.0,878.0,m,0,0,32012.0,64000.0,716.0


In [7]:
del df_raw_1, df_raw_2, df_raw, df_csv

In [8]:
df = df_all[~df_all.gender.isna()]
df['gender_dum'] = 1
df.loc[df.gender == 'f', 'gender_dum'] = 0
df = df.drop('gender', axis=1).astype({'approved_dum': 'int64', 'reward': 'int64'})
df['age_dum'] = 1
df.loc[df.age < 28, 'age_dum'] = 0
df

,customer_id,loan_request_id,age,connections,apps,sms,contacts,reward,approved_dum,salary,loan_amount,CIBIL,gender_dum,age_dum
0,150,218,32.0740,811.0,10.0,584.0,1820.0,1,1,40000.0,5000.0,707.0,1,1
1,150,97964,32.0740,811.0,10.0,584.0,1820.0,-1,1,40000.0,40000.0,707.0,1,1
2,150,66,32.0740,811.0,10.0,584.0,1820.0,1,1,40000.0,5000.0,707.0,1,1
3,150,2312,32.0740,811.0,10.0,584.0,1820.0,1,1,40000.0,5000.0,707.0,1,1
4,150,3754,32.0740,811.0,10.0,584.0,1820.0,1,1,40000.0,5000.0,707.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261924,1600000,282451,18.3178,1.0,24.0,121.0,171.0,0,0,115000.0,200000.0,747.0,1,0
261958,1600000,283249,30.0904,736.0,57.0,1638.0,862.0,0,0,50251.0,101000.0,663.0,1,1
261970,1600000,283473,18.6904,196.0,37.0,1328.0,203.0,0,0,68328.0,137000.0,745.0,1,0
261987,1600000,283850,27.9096,9.0,48.0,237.0,878.0,0,0,32012.0,64000.0,716.0,1,0


In [9]:
df_small = df.drop(['customer_id', 'loan_request_id', 'age', 'CIBIL', 'loan_amount'], axis=1)
log_vars = ['salary', 'connections',  'apps', 'sms', 'contacts']
for c in log_vars:
    df_small[c] = np.log(df_small[c] + 1)
norm_vars = log_vars
scaler = StandardScaler().fit(df_small[norm_vars])
df_small[norm_vars] = scaler.transform(df_small[norm_vars])
df_small['dum'] = 0
df_small.loc[(df_small.age_dum == 0) & (df_small.gender_dum == 1), 'dum'] = 1
df_small.loc[(df_small.age_dum == 1) & (df_small.gender_dum == 0), 'dum'] = 2
df_small.loc[(df_small.age_dum == 1) & (df_small.gender_dum == 1), 'dum'] = 3
df_small

,connections,apps,sms,contacts,reward,approved_dum,salary,gender_dum,age_dum,dum
0,0.742061,-2.373794,-0.045504,0.890059,1,1,0.432050,1,1,3
1,0.742061,-2.373794,-0.045504,0.890059,-1,1,0.432050,1,1,3
2,0.742061,-2.373794,-0.045504,0.890059,1,1,0.432050,1,1,3
3,0.742061,-2.373794,-0.045504,0.890059,1,1,0.432050,1,1,3
4,0.742061,-2.373794,-0.045504,0.890059,1,1,0.432050,1,1,3
...,...,...,...,...,...,...,...,...,...,...
261924,-2.702040,-1.024403,-0.808190,-0.729581,0,0,2.584841,1,0,1
261958,0.686491,0.358826,0.455738,0.377513,0,0,0.897138,1,1,3
261970,-0.070057,-0.336196,0.353732,-0.612465,0,0,1.523554,1,0,1
261987,-1.779172,0.081672,-0.483064,0.390122,0,0,-0.022064,1,0,1


In [10]:
np.random.seed(1)

y = df_small.approved_dum.values.reshape(-1, 1)
r = df_small.reward.values.reshape(-1, 1)
s = df_small.dum.values.reshape(-1, 1)
a = df_small.drop([
    'approved_dum', 'age_dum', 'gender_dum', 'dum', 'reward',
], axis=1).values

n_test = 190000

shuffle_idx = np.arange(y.shape[0])
np.random.shuffle(shuffle_idx)
y_train, y_test = y[shuffle_idx[n_test:]], y[shuffle_idx[:n_test]]
r_train, r_test = r[shuffle_idx[n_test:]], r[shuffle_idx[:n_test]]
s_train, s_test, s_eval = s[shuffle_idx[n_test:]], s[shuffle_idx[:n_test]], np.array([0, 1] * n_test).reshape(-1, 1)
a_train, a_test, a_eval = a[shuffle_idx[n_test:]], a[shuffle_idx[:n_test]], np.repeat(a[shuffle_idx[:n_test]], 2, axis=0)

In [11]:
df_small_opt = FairOptimization(s_train, a_train, y_train, r_train)

In [12]:
df_small_opt.ftup.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                13656
Model:                          Logit   Df Residuals:                    13650
Method:                           MLE   Df Model:                            5
Date:                Wed, 04 Nov 2020   Pseudo R-squ.:                 0.02471
Time:                        10:52:06   Log-Likelihood:                -3232.7
converged:                       True   LL-Null:                       -3314.6
Covariance Type:            nonrobust   LLR p-value:                 1.520e-33
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.7321      0.037     74.022      0.000       2.660       2.804
x1             0.1985      0.034      5.832      0.000       0.132       0.265
x2             0.2559      0.035      7.369      0.000       0.188       0.324
x3             0.1999      0.034      5.853      0.000       0.133       0.267
x4            -0.1066      0.042     -2.525      0.012      -0.189      -0.024
x5            -0.1345      0.035     -3.872      0.000      -0.203      -0.066
==============================================================================
"""

In [13]:
optimization = df_small_opt.optimize(
    df_small_opt.ipwe, 
    method='shgo', 
    bounds=[(2, 3), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1)]
)

In [14]:
optimization.x

array([ 3., -1., -1., -1.,  1.,  1.])

In [15]:

eval = df_small_opt.evaluate(eta=optimization.x, s_test=s_test, a_test=a_test, y_test=y_test, r_test=r_test, metrics=['cf', 'mae', 'er'])
pd.DataFrame(eval, index=['CF', 'MAE', 'E(R)'], columns=['y_ml', 'y_rml', 'y_ftu', 'y_aa', 'y_1', 'y_2', 'y_eta'])

,y_ml,y_rml,y_ftu,y_aa,y_1,y_2,y_eta
CF,0.035571,0.015009,0.010922,0.001078,0.000757,0.000733,0.008621
MAE,0.123060,0.096225,0.124085,0.124369,0.123739,0.123746,0.181904
E(R),0.976769,0.952890,0.977892,0.978053,0.977842,0.977804,1.028077


In [16]:
getattr(df_small_opt, 'ipwe')

<bound method FairOptimization.ipwe of <fairopt.FairOptimization object at 0x7f62bad4f0a0>>